In [1]:
import os
import random
import torch
import numpy as np
import pickle as pkl
from analysis import *
import argparse
from sys import platform

In [2]:
torch.manual_seed(12)
torch.cuda.manual_seed(12)
np.random.seed(12)
torch.backends.cudnn.deterministics = True
torch.set_num_threads(1)

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())


print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
NVIDIA TITAN V


In [18]:
# iv: image option
length = 440
channel = 128
min_CNN = 200
n_classes = 40
classes = range(n_classes)


In [4]:
if platform == "linux" or platform == "linux2":
    torch_models_dir = r"/media/titan/AI Research1/Data/CVPR2017"
elif platform == "win32":
    torch_models_dir = r"D:\Data\CVPR2021-02785\CVPR2021-02785\preprocessed\torch_models"
block_splits_all, block_splits_single, eeg_14_70, eeg_55_95, eeg_5_95, eeg_raw = os.listdir(torch_models_dir)
print(os.listdir(torch_models_dir))

['block_splits_by_image_all.pth', 'block_splits_by_image_single.pth', 'eeg_14_70_std.pth', 'eeg_55_95_std.pth', 'eeg_5_95_std.pth', 'eeg_signals_raw_with_mean_std.pth']


In [5]:
eeg_dataset = os.path.join(torch_models_dir, eeg_5_95)
splits_all_path = os.path.join(torch_models_dir, block_splits_all)
splits_single_path = os.path.join(torch_models_dir, block_splits_single)
# splits_path = os.path.join(torch_models_dir, splits_shuffled_path)
print(eeg_dataset,'\n', splits_all_path, '\n', splits_single_path)

/media/titan/AI Research1/Data/CVPR2017/eeg_5_95_std.pth 
 /media/titan/AI Research1/Data/CVPR2017/block_splits_by_image_all.pth 
 /media/titan/AI Research1/Data/CVPR2017/block_splits_by_image_single.pth


In [6]:
splits_all = torch.load(splits_all_path)
splits_single = torch.load(splits_single_path)

In [7]:
print(len(splits_all['splits']))
print(len(splits_all['splits'][0]))

print(len(splits_all['splits'][5]['train']))
print(len(splits_all['splits'][5]['val']))
print(len(splits_all['splits'][5]['test']))
print(splits_all['splits'][0]['train'][:40])
print(splits_all['splits'][1]['train'][:40])
print(splits_all['splits'][2]['train'][:10])
print(splits_all['splits'][3]['train'][:10])


6
3
7984
1996
1985
[0, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 27, 29, 33, 35, 36, 37, 38, 39, 42, 43, 44, 45, 46, 47, 48, 51, 52, 54, 55]
[1, 2, 3, 4, 6, 8, 9, 12, 13, 20, 25, 26, 27, 28, 30, 32, 33, 35, 37, 38, 39, 40, 44, 45, 46, 50, 52, 54, 56, 58, 59, 60, 62, 65, 68, 72, 73, 74, 76, 81]
[2, 3, 4, 5, 6, 7, 8, 10, 11, 13]
[1, 2, 4, 7, 9, 10, 12, 13, 14, 15]


In [8]:
# print(splits_single)
print(len(splits_single['splits'][0]['train']))
print(len(splits_single['splits'][0]['val']))
print(len(splits_single['splits'][0]['test']))

669
167
164


In [9]:
eeg_loaded = torch.load(eeg_dataset)

In [10]:
print(len(eeg_loaded))
print(eeg_loaded.keys())
dataset, labels, images = [eeg_loaded[k] for k in eeg_loaded.keys()]
print(len(labels))
print(len(images))
print(len(dataset))

print(labels)
print(images[0])
print(dataset[0]['eeg'].shape)
print(dataset[0])

3
dict_keys(['dataset', 'labels', 'images'])
40
1996
11965
['n02389026', 'n03888257', 'n03584829', 'n02607072', 'n03297495', 'n03063599', 'n03792782', 'n04086273', 'n02510455', 'n11939491', 'n02951358', 'n02281787', 'n02106662', 'n04120489', 'n03590841', 'n02992529', 'n03445777', 'n03180011', 'n02906734', 'n07873807', 'n03773504', 'n02492035', 'n03982430', 'n03709823', 'n03100240', 'n03376595', 'n03877472', 'n03775071', 'n03272010', 'n04069434', 'n03452741', 'n03792972', 'n07753592', 'n13054560', 'n03197337', 'n02504458', 'n02690373', 'n03272562', 'n04044716', 'n02124075']
n02951358_31190
torch.Size([128, 500])
{'eeg': tensor([[-0.0098,  0.0195,  0.0620,  ...,  0.0638,  0.0120, -0.0118],
        [-0.0045,  0.1303,  0.2673,  ...,  0.0894,  0.0342, -0.0082],
        [ 0.0215, -0.2017, -0.4305,  ..., -0.2022, -0.0940,  0.0188],
        ...,
        [ 0.0160,  0.0707,  0.1005,  ...,  0.2066,  0.1156,  0.0036],
        [-0.0046, -0.0084, -0.0119,  ...,  0.0007, -0.0026, -0.0053],
        [ 

In [11]:
opt = {
    # Dataset options
#     "iv": "image",
#     "offset": None,
    "results_file": "results.pkl",
    "subject": 0,
    "time_low": 20,
    "time_high": 460,
#     "run": "none",
    "eeg_dataset": eeg_dataset,
    "model_type": "model10",
    "splits_path": splits_all_path,
    "split_num": 0,
    "split_name": "train",
#     "fold": 5,
    #Training options
    "batch_size": 16,
    "optim": "Adam",
    "learning_rate": 0.001,
    "learning_rate_decay_by": 0.5,
    "learning_rate_decay_every": 10,
    "epochs": 100,
    "GPUindex": 0,
    "kind":"from-scratch",
    #Backend options
    "no_cuda": False,
    "classifier": None
}
opt = argparse.Namespace(**opt)
print(opt.time_low)

20


In [12]:
%load_ext autoreload
%autoreload 2
from torch.utils.data import DataLoader
# from data_loader import EEGDataset, Splitter, SplitterWithData
from data_loader_CVPR2017 import EEGDataset, Splitter
from EEG_Encoder.LSTM import classifier_LSTM
from EEG_Encoder.CNN import classifier_CNN
from EEG_Encoder.EEGNet import classifier_EEGNet
from EEG_Encoder.SyncNet import classifier_SyncNet
from EEG_Encoder.EEGChannelNet import classifier_EEGChannelNet
from EEG_Encoder.net_generator import Classifier
from EEG_Encoder.net_trainer import net_trainer
from p_values import *
from torchinfo import summary

In [13]:
def load_dataset(
#              offset,
             eeg_dataset,
             splits_path,
             split_num, # (0-5) - 6 fold cross validation
             split_name,
#              total, 
#              classes,
#              classifier,
             batch_size,
#              GPUindex,
#              length, # 500
#              channel, # 128
#              min_CNN,
             opt,
             kind="from-scratch"):        
    # Load dataset
    dataset = EEGDataset(opt, eeg_dataset)
    print("DONE: LOAD DATASET")
#     # Create loaders for LSTM/MLP/CNN/SCNN/EEGNet/SyncNet/EEGChannelNet
#     if kind=="from-scratch":
#         relabel = False
#     if kind=="incremental":
#         relabel = False
#     if kind=="no-model-file":
#         relabel = True
    splitter = {split: Splitter(dataset,
                    splits_path,
                    split_num,
                    split_name) for split in ["train", "val", "test"]}
    loaders = {split: DataLoader(
                        splitter[split],
                        batch_size = batch_size,
                        drop_last = False,
                        shuffle = True)
                    for split in ["train", "val", "test"]}
    channel_idx = None    
    print("DONE: Create loaders for model")            
    return dataset, loaders, splitter

In [14]:
# Options
opt.classifier = "EEGNet"
opt.batch_size = 16
# opt.kind = "from-scratch"
# opt.run = "imagenet40-1000"
# opt.fold = 5

In [15]:
dataset, loaders, splitter = load_dataset(
#              offset,
             opt.eeg_dataset,
             opt.splits_path,
             opt.split_num, # (0-5) - 6 fold cross validation
             opt.split_name,
#              total, 
#              classes,
#              classifier,
             opt.batch_size,
#              GPUindex,
#              length, # 500
#              channel, # 128
#              min_CNN,
             opt,
             opt.kind)

DONE: LOAD DATASET
DONE: Create loaders for model


In [16]:
# loaders: divide the splits data in each fold with batch_size
# Each fold has {train: 8000 idx, val: 2000 idx, test: 2000 idx}
# Each loader batch has {train: 2000 idx, val: 250 idx, test: 250 idx}
print(type(dataset))
print(type(loaders))
print(len(loaders), [len(loaders[name]) for name in ["train", "val", "test"] ])
for i, (input, target) in enumerate(loaders["train"]):
    if i<20:
        print(f"{i+1}: Target size: {target.size()}; input size: {input.size()}")
# for i in range(0, 40):
#     eeg, label_val = splitter["val"][i]
#     eeg, label_train = splitter["train"][i]
#     print(f"{i+1}: Label val: {label_val}; label train: {label_train}")


<class 'data_loader_CVPR2017.EEGDataset'>
<class 'dict'>
3 [498, 498, 498]
1: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
2: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
3: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
4: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
5: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
6: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
7: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
8: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
9: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
10: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
11: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
12: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
13: Target size: torch.Size([16]); input size:

In [19]:
net, nonclasses = Classifier(
                 n_classes,
                 classes,
                 opt.classifier,
                 opt.GPUindex,
                 length,
                 channel,
                 min_CNN,
                 opt.kind)
# print(len(nonclasses))
summary(net, input_size=(1,128, 440))

DONE: CREATE TORCH CLASSIFIER
classifier_EEGNet(
  (network): Sequential(
    (0): ZeroPad2d((110, 109, 0, 0))
    (1): Conv2d(1, 8, kernel_size=(1, 220), stride=(1, 1))
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(8, 16, kernel_size=(128, 1), stride=(1, 1), groups=8)
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ELU(alpha=1.0)
    (6): AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0)
    (7): Dropout(p=0.5, inplace=False)
    (8): ZeroPad2d((8, 7, 0, 0))
    (9): Conv2d(16, 16, kernel_size=(1, 16), stride=(1, 1), groups=8)
    (10): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
    (11): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ELU(alpha=1.0)
    (13): AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0)
    (14): Dropout(p=0.5, inplace=False)
  )
  (fc): Linear(in_features=208, out_features=40, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
classifier_EEGNet                        [1, 40]                   --
├─Sequential: 1-1                        [1, 16, 1, 13]            --
│    └─ZeroPad2d: 2-1                    [1, 1, 128, 659]          --
│    └─Conv2d: 2-2                       [1, 8, 128, 440]          1,768
│    └─BatchNorm2d: 2-3                  [1, 8, 128, 440]          16
│    └─Conv2d: 2-4                       [1, 16, 1, 440]           2,064
│    └─BatchNorm2d: 2-5                  [1, 16, 1, 440]           32
│    └─ELU: 2-6                          [1, 16, 1, 440]           --
│    └─AvgPool2d: 2-7                    [1, 16, 1, 110]           --
│    └─Dropout: 2-8                      [1, 16, 1, 110]           --
│    └─ZeroPad2d: 2-9                    [1, 16, 1, 125]           --
│    └─Conv2d: 2-10                      [1, 16, 1, 110]           528
│    └─Conv2d: 2-11                      [1, 16, 1, 110]           272
│    └─

In [20]:
model_path = (   opt.classifier+
                  "-"+
                  str(length)+
                  "-"+
                  str(channel) )
print(model_path)

EEGNet-440-128


In [21]:
print(opt)

Namespace(results_file='results.pkl', subject=0, time_low=20, time_high=460, eeg_dataset='/media/titan/AI Research1/Data/CVPR2017/eeg_5_95_std.pth', model_type='model10', splits_path='/media/titan/AI Research1/Data/CVPR2017/block_splits_by_image_all.pth', split_num=0, split_name='train', batch_size=16, optim='Adam', learning_rate=0.001, learning_rate_decay_by=0.5, learning_rate_decay_every=10, epochs=100, GPUindex=0, kind='from-scratch', no_cuda=False, classifier='EEGNet')


In [22]:
channel_idx=None
non_classes=None
if opt.kind=="from-scratch":
    loss_history, accuracy_val, accuracy_test = net_trainer(
            net,
            loaders,
            opt,
            channel_idx,
            nonclasses,
            None,
            True,
            model_path)

epoch 1
Size of input:  torch.Size([16, 128, 440])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 100: Loss=3.7824759483337402; accuracy=0.026249999180436134
Batch 200: Loss=3.7356367111206055; accuracy=0.0228124987334013
Batch 300: Loss=3.753798007965088; accuracy=0.02395833469927311
Batch 400: Loss=3.715764045715332; accuracy=0.025624999776482582
Validation accuracy: 0.056640625
epoch 2
Size of input:  torch.Size([16, 128, 440])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 100: Loss=3.617763042449951; accuracy=0.054375000298023224
Batch 200: Loss=3.342247247695923; accuracy=0.05374999716877937
Batch 300: Loss=3.418456554412842; accuracy=0.052916668355464935
Batch 400: Loss=3.360786199569702; accuracy=0.05406250059604645
Validation accuracy: 0.07421875
epoch 3
Size of input:  torch.Size([16, 128, 440])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 100: Loss=3.2971982955932617; accuracy=0.06

epoch 21
Size of input:  torch.Size([16, 128, 440])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 100: Loss=2.3545258045196533; accuracy=0.2537499964237213
Batch 200: Loss=2.228386402130127; accuracy=0.25312498211860657
Batch 300: Loss=2.032985210418701; accuracy=0.25187501311302185
Batch 400: Loss=1.8208569288253784; accuracy=0.25296875834465027
Validation accuracy: 0.361328125
epoch 22
Size of input:  torch.Size([16, 128, 440])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 100: Loss=2.4822587966918945; accuracy=0.25
Batch 200: Loss=2.0991225242614746; accuracy=0.24781249463558197
Batch 300: Loss=2.2066333293914795; accuracy=0.25041666626930237
Batch 400: Loss=1.9286632537841797; accuracy=0.24843749403953552
Validation accuracy: 0.443359375
epoch 23
Size of input:  torch.Size([16, 128, 440])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 100: Loss=2.1720409393310547; accuracy=0.2606250047683

epoch 41
Size of input:  torch.Size([16, 128, 440])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 100: Loss=2.331191062927246; accuracy=0.32499998807907104
Batch 200: Loss=1.7497458457946777; accuracy=0.32249999046325684
Batch 300: Loss=1.8174577951431274; accuracy=0.3239583373069763
Batch 400: Loss=1.8115663528442383; accuracy=0.32640624046325684
Validation accuracy: 0.515625
epoch 42
Size of input:  torch.Size([16, 128, 440])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 100: Loss=2.381694793701172; accuracy=0.3362500071525574
Batch 200: Loss=1.474516749382019; accuracy=0.3462499976158142
Batch 300: Loss=1.9314762353897095; accuracy=0.3454166650772095
Batch 400: Loss=2.9367125034332275; accuracy=0.34171873331069946
Validation accuracy: 0.45703125
epoch 43
Size of input:  torch.Size([16, 128, 440])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 100: Loss=1.5675514936447144; accuracy=0.335624

Validation accuracy: 0.46875
epoch 61
Size of input:  torch.Size([16, 128, 440])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 100: Loss=1.510969638824463; accuracy=0.38374999165534973
Batch 200: Loss=1.509389877319336; accuracy=0.38374999165534973
Batch 300: Loss=2.1790871620178223; accuracy=0.3791666626930237
Batch 400: Loss=1.9724500179290771; accuracy=0.37593749165534973
Validation accuracy: 0.53125
epoch 62
Size of input:  torch.Size([16, 128, 440])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 100: Loss=1.6196216344833374; accuracy=0.3774999976158142
Batch 200: Loss=1.8566440343856812; accuracy=0.37687498331069946
Batch 300: Loss=2.1192307472229004; accuracy=0.3764583468437195
Batch 400: Loss=2.048961877822876; accuracy=0.37812498211860657
Validation accuracy: 0.51953125
epoch 63
Size of input:  torch.Size([16, 128, 440])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 100: Loss=1.316421

Validation accuracy: 0.48828125
epoch 81
Size of input:  torch.Size([16, 128, 440])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 100: Loss=1.0287106037139893; accuracy=0.40625
Batch 200: Loss=1.4953469038009644; accuracy=0.4046874940395355
Batch 300: Loss=1.4389139413833618; accuracy=0.4047916829586029
Batch 400: Loss=1.9796888828277588; accuracy=0.40296873450279236
Validation accuracy: 0.560546875
epoch 82
Size of input:  torch.Size([16, 128, 440])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 100: Loss=1.4613871574401855; accuracy=0.3943749964237213
Batch 200: Loss=1.9155244827270508; accuracy=0.3921875059604645
Batch 300: Loss=1.2456332445144653; accuracy=0.3920833468437195
Batch 400: Loss=1.53557288646698; accuracy=0.3935937285423279
Validation accuracy: 0.5390625
epoch 83
Size of input:  torch.Size([16, 128, 440])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 100: Loss=1.85915589332580

Validation accuracy: 0.548828125


In [23]:
val =accuracy_val
test = accuracy_test

print("Validation accuracy: ", val)
print("Test accuracy: ", test)

Validation accuracy:  0.5623822212219238
Test accuracy:  0


In [24]:
print(accuracy_val)

0.5623822212219238
